## Cellpose Comparisons
As a test run, I will just evaluate the performance on the training dataset (or maybe a random subset of it) using two networks with/without the skel flag. 

In [ ]:
# load images - reuse code from mask checking, load masks too for first comparison 
# define network directories
# run the network on a random set from the list of images 
# define and run comparison metrics 
#additionally, figure out how to select regions with poor scores 

%load_ext autoreload
%autoreload 2


#display ground truth images, overlay, and flow 
# basedir = '/home/kcutler/DataDrive/skelpose_train/registered/Non-Ensemble/'
# basedir = '/home/kcutler/DataDrive/skelpose_test'
basedir = '/home/kcutler/DataDrive/skelpose_all/test'
# basedir = '/home/kcutler/DataDrive/crop/'
from pathlib import Path
from cellpose_omni import plot, models, utils, io, dynamics
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import time, os, sys


In [ ]:
mask_filter = '_masks'
img_names = io.get_image_files(basedir,mask_filter,look_one_level_down=True)
mask_names,flow_names = io.get_label_files(img_names, mask_filter)
imgs = [skimage.io.imread(f) for f in img_names]
masks_gt = [skimage.io.imread(f) for f in mask_names]
# flows_gt = [skimage.io.imread(f) for f in flow_names]

In [ ]:
cnt = 0;
for mask in masks_gt:
    lbls = np.unique(utils.format_labels(mask))
    cnt += len(lbls[lbls>0])
print(cnt)

In [ ]:
from cellpose_omni import models

use_GPU = models.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

modeldir_orig = '/home/kcutler/DataDrive/skelpose_train/registered/models/cellpose_residual_on_style_on_concatenation_off_registered_2021_07_22_11_48_00.465748' #500 epochs, bs 8, lr .2
modeldir_orig_1000 =  '/home/kcutler/DataDrive/skelpose_train/registered/models/cellpose_residual_on_style_on_concatenation_off_registered_2021_07_22_14_12_00.303825' #1000 epochs
modeldir_orig_2000 =  '/home/kcutler/DataDrive/skelpose_train/registered/models/cellpose_residual_on_style_on_concatenation_off_registered_2021_07_22_15_52_12.716108'

# something to keep in mind... there was that GPU bug, but I think it was using CPU when generating the ground truth data? need to check...
modeldir_interp = '/home/kcutler/DataDrive/skelpose_train/registered/models/cellpose_residual_on_style_on_concatenation_off_registered_2021_07_21_00_08_37.585178_epoch_3951'
md_int_base = '/home/kcutler/DataDrive/skelpose_train/registered/models/cellpose_residual_on_style_on_concatenation_off_registered_2021_07_21_00_08_37.585178_epoch_'

#latest but with interpolated filed, so dark flow at divergent points 

model = [models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir_orig,nclasses=3),
         models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir_orig_1000,nclasses=3),
         models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir_orig_2000,nclasses=3),
         models.CellposeModel(gpu=use_GPU, pretrained_model=(md_int_base+'501'),nclasses=4),
         models.CellposeModel(gpu=use_GPU, pretrained_model=(md_int_base+'1001'),nclasses=4),
         models.CellposeModel(gpu=use_GPU, pretrained_model=(md_int_base+'2001'),nclasses=4),
         models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir_interp,nclasses=4)]

chans = [0,0] 

In [ ]:
names = ['Original-Cellpose_500','Original-Cellpose_1000','Original-Cellpose_2000','Frankenpose_500',
         'Skelpose-latest-interp_500','Skelpose-latest-interp_1000','Skelpose-latest-interp_2000',
         'Skelpose-latest-interp_4000']

In [ ]:
imglist = imgs[:]
n = len(names)
N = len(imgs)
masks,flows,styles,d = [[]]*n,[[]]*n,[[]]*n,[[]]*n

In [ ]:
# masks[1], flows[1], styles[1] = model[1].eval(imglist,channels=chans,rescale=None,dist_threshold=-1,flow_threshold=0,skel=False,resample=False,tile=False)
# masks[2], flows[2], styles[2] = model[2].eval(imglist,channels=chans,rescale=None,dist_threshold=-1,flow_threshold=0,skel=False,resample=False,tile=False)
for j in [4,5,6]:
    masks[j], flows[j], styles[j] = model[j-1].eval(imglist,channels=chans,rescale=None,dist_threshold=-1,flow_threshold=0,skel=True,resample=False,tile=False)

# original, no skel reconstruction 
# masks[0], flows[0], styles[0] = model[0].eval(imglist,channels=chans,rescale=None,dist_threshold=-1,flow_threshold=0,skel=False,resample=False,tile=False)
# original + skel 
# masks[1], flows[1], styles[1] = model[0].eval(imglist,channels=chans,rescale=None,dist_threshold=-1,flow_threshold=0,skel=True,resample=False,tile=False)
# new model+skel
# masks[2], flows[2], styles[2] = model[1].eval(imglist,channels=chans,rescale=None,dist_threshold=-1,flow_threshold=0,skel=True,resample=False,tile=False)

In [ ]:
# for j in [1,2]:
for j in [4,5,6]:
# for j in range(n):
    savedir = '/home/kcutler/DataDrive/skelpose_paper/Comparison Examples/'+names[j]
    if not os.path.isdir(savedir):
        os.mkdir(savedir)
    io.save_masks(imgs, masks[j], flows[j], img_names,save_flows=True,save_outlines=True,savedir=savedir)
#     io.save_masks(imgs, masks[j], flows[j], img_names,save_outlines=True,savedir=savedir)

In [ ]:
import os
for j in range(n):
    savedir = '/home/kcutler/DataDrive/skelpose_paper/Comparison Examples/'+names[j]
    mask_names = [os.path.join(savedir+'/masks',os.path.splitext(os.path.basename(name))[0]+'_cp_masks.png') for name in img_names]
    flow_names = [os.path.join(savedir+'/flows',os.path.splitext(os.path.basename(name))[0]+'_flows.png') for name in img_names]
    masks[j] = [utils.format_labels(utils.clean_boundary(skimage.io.imread(f))) for f in mask_names]
    flows[j] = [skimage.io.imread(f) for f in flow_names]

In [ ]:
from cellpose_omni import metrics
from skimage import measure
threshold=np.linspace(0.5,1,100)
import fastremap
# ap,tp,fp,fn  = map(list,zip(*[ metrics.average_precision([masks_gt[0]],masks_pred,threshold=threshold) for masks_pred in masks]))
# ap = np.zeros((100,len(masks)))
# IoU = np.zeros((len(masks),len(masks_gt)))

#eventually look at overlaps of predited that are above x percent their size 

In [ ]:
# This code ignores image boundary cells. 

ap = [[]] * len(masks) # average precision matrix per image
tp = [[]] * len(masks)
fp = [[]] * len(masks)
fn = [[]] * len(masks)
IoU = [[]] * len(masks) # Intersection over Union 
OvR = [[]] * len(masks)
pred_areas = [[]] * len(masks)

nimg = len(masks_gt)
cell_areas = [[]] * nimg
masks_gt_clean = [None]*nimg
masks_pred_clean = [[None]*nimg]*len(masks)
# remapping = [[]] * nimg
for j in range(nimg):
    mgt = utils.format_labels(utils.clean_boundary(masks_gt[j]))
    masks_gt_clean[j] = mgt
    regions = measure.regionprops(mgt)
    areas = np.array([reg.area for reg in regions])
    cell_areas[j] =  areas

# go over each model
for j,masks_pred in enumerate(masks):
#     masks_pred = map(list,zip(*[ utils.format_labels(utils.clean_boundary(msk)) for msk in masks_pred]))
# just apply cleanup to the masks when reading them in 
    api,tpi,fpi,fni = metrics.average_precision(masks_gt_clean,masks_pred,threshold=threshold)
    ap[j] = ap[j]+[api]
    tp[j] = tp[j]+[tpi]
    fp[j] = fp[j]+[fpi]
    fn[j] = fn[j]+[fni]
    

    
    masks_pred_clean[j] = masks_pred
    # go over every image
    
    for k in range(nimg):
        # get the IoU matrix; axis 0 corresponds to GT, axis 1 to pred 

        regions = measure.regionprops(masks_pred[k])
        areas = np.array([reg.area for reg in regions])
        pred_areas[j] = pred_areas[j] + [areas]
        iou = metrics._intersection_over_union(masks_gt_clean[k], masks_pred[k])
        ovp = metrics._label_overlap(masks_gt_clean[k], masks_pred[k])[1:,1:] #throw out columns corresponding to zero  
#         tp = metrics._true_positive(iou, th)
        OvR[j] = OvR[j]+[ovp / areas[np.newaxis,:]] # Overlap Ratio           
        IoU[j] = IoU[j]+[iou]
#         seg_error_percent[j] = 
 

In [ ]:
savedir = '/home/kcutler/DataDrive/skelpose_paper/Comparison Examples/'

In [ ]:
np.savez(savedir+'OvR',OvR)
np.savez(savedir+'IoU',IoU)
np.savez(savedir+'cell_areas',cell_areas)
np.savez(savedir+'remapping',remapping)
np.savez(savedir+'ap',ap)

In [ ]:
OvR = np.load(savedir+'OvR'+'.npz',allow_pickle=True)['arr_0']
IoU = np.load(savedir+'IoU'+'.npz',allow_pickle=True)['arr_0']
cell_areas = np.load(savedir+'cell_areas'+'.npz',allow_pickle=True)['arr_0']
remapping = np.load(savedir+'remapping'+'.npz',allow_pickle=True)['arr_0']
ap = np.load(savedir+'ap'+'.npz',allow_pickle=True)['arr_0']

In [ ]:

N = len(names)
nimg = len(cell_areas)
cell_errors = [[]]*N
total_errors = [0]*N
total_single_errors = [0]*N
total_cells = len(np.concatenate(cell_areas))

# area_thresh = np.linspace(1,np.max(np.concatenate(cell_areas)),100)
area_thresh = [np.percentile(np.concatenate(cell_areas),i) for i in range(100)]
M = len(area_thresh)
# ce_thresh = [[[]]*M]*N
# te_thresh = [[0]*M]*N
# tse_thresh = [[0]*M]*N
# tc_thresh = [0]*M

percent = [0]*N

for j in range(N):
# for j in [-1,-2]:
    
    print(names[j])
    ce_thresh = [[]]*M
    te_thresh = [0]*M
    tse_thresh = [0]*M
    tc_thresh = [0]*M
    
    for k in range(nimg):
        r = OvR[j][k].copy() #overlap ratio 
        r[r<=0.75] = 0 
        mx = np.max(r,axis=0) #find the maximum overlap for each true cell 
        mx[mx==0] = np.nan #exclude case where a spurious label has no overlaps
        hits = np.sum(r==mx,axis=1) # sum will be zero if a cell label has zero hits, not sure which ones these are
        cell_errors[j] = cell_errors[j] + [np.abs(hits-1)] #error if >1 or =0 
        total_errors[j] += np.sum(hits[hits>1]-1)+np.sum(hits==0) # -1 because a hit of 2 is 2 pred labels, 1 'extra' or 1 error
        total_single_errors[j] += np.sum(hits>1)+np.sum(hits==0)
        
        ca = cell_areas[k]
        for i,a in enumerate(area_thresh):
            cell_filter = ca>=a
            cell_count = np.count_nonzero(cell_filter)

            hits_thresh = hits[cell_filter] # sum will be zero if a cell label has zero hits, not sure which ones these are
            ce_thresh[i] += [np.abs(hits_thresh-1)] #error if >1 or =0 
            te_thresh[i] += np.sum(hits_thresh[hits_thresh>1]-1)+np.sum(hits_thresh==0) # -1 because a hit of 2 is 2 pred labels, 1 'extra' or 1 error
            tse_thresh[i] += np.sum(hits_thresh>1)+np.sum(hits_thresh==0)
            tc_thresh[i] += cell_count
    
#     for i in range(M):
#         print('percentage of cells with at least one error above cutoff:', tse_thresh[i]/tc_thresh[i] *100)
        
    percent[j] = [tse_thresh[i]/tc_thresh[i]*100 for i in range(M)]
        
    
    print('overall percentage of cells with at least one error:',total_single_errors[j]/total_cells *100)
#     print('percentage of cells with at least one error above cutoff:', tse_thresh[j]/tc_thresh *100)

    

In [ ]:
pretty_names = ['Original Cellpose','','','Mixed Method','','','','New Method']


def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

from cellpose_omni.utils import sinebow
n = len(names)
master_color_scheme = [[i,0,0] for i in np.linspace(1,.5,3)]+[[i,i,i] for i in np.linspace(.75,0,n-3)]

golden = (1 + 5 ** 0.5) / 2
sz = 2.5
labelsize = 7

%matplotlib inline
darkmode = False
if darkmode:
    plt.style.use('dark_background')
    axcol = 'w'
    colors = sinebow(n+1)
    colors = [colors[j+1] for j in range(n)]
    background_color = 'k'
    suffix = '_dark'
else:
    mpl.rcParams.update(mpl.rcParamsDefault)
    axcol = 'k'
    cmap = mpl.cm.get_cmap('viridis')
#     colors = cmap(np.linspace(0,.9,len(names)))
    colors = master_color_scheme
    background_color = np.array([1,1,1,0])
    suffix = ''
    
mpl.rcParams['figure.dpi'] = 300

# could turn these results into a plot by repeating at different cutoffs 
fig = plt.figure(figsize=(sz, sz/golden)) 
ax = plt.axes()
# for j in range(N):
for j in [0,3,-1]:
#     ax.plot(area_thresh,np.divide(tse_thresh[j],tc_thresh),label=names[j],color=colors[j])
#     ax.plot(range(100),percent[j],label=names[j],color=colors[j])
    ax.plot(range(100),percent[j],label=pretty_names[j],color=colors[j])
#     print('mean percent error is',np.mean(percent[j]))
    # ax.set_facecolor('w')
# plt.xscale('log')
# ax.legend(prop={'size': labelsize}, loc='upper left', frameon=False,bbox_to_anchor=(1.05, 1))
ax.legend(prop={'size': labelsize}, loc='best', frameon=False)
ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.tick_params(axis='both', which='minor', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.set_ylabel('Percent of cells \nwith >=1 error', fontsize = labelsize)
ax.set_xlabel('Area percentile cutoff', fontsize = labelsize)
plt.ylim([0,100])


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.patch.set_alpha(0.0)


fig.patch.set_facecolor(background_color)
plt.show()

a = 35
tight_bbox_raw = ax.get_tightbbox(fig.canvas.get_renderer())
tight_bbox_raw._points+=[[-a,-a],[a,a]]
tight_bbox = mpl.transforms.TransformedBbox(tight_bbox_raw, mpl.transforms.Affine2D().scale(1./fig.dpi))


name = '/home/kcutler/DataDrive/skelpose_paper/error_vs_area_percentiles'+suffix
fig.savefig(name+'.eps',bbox_inches=tight_bbox)
fig.savefig(name+'.png',bbox_inches=tight_bbox)
# fig.savefig(name+'.eps',pad_inches = 0.5)
# fig.savefig(name+'.png',pad_inches = 0.5)

In [ ]:
x = threshold

fig = plt.figure(figsize=(sz, sz/golden)) 
ax = plt.axes()
# plt.tight_layout()
# plt.minorticks_on()
plt.xticks(np.arange(min(x), max(x)+1, .25))
plt.xlim([0.45,1.05])
plt.ylim([0,1])
# plt.yticks(np.arange(0, 1.1, .25))

# colors = ['g','r','b','y','c','m']
cell_count = np.array([len(np.unique(ca)) for ca in cell_areas])
# for j in range(n):
for j in [0,3,-1]:
    ax.plot(x,np.mean(ap[j][0],axis=0).T,label=pretty_names[j],color=colors[j])
#     ax.plot(x,np.mean((fp[j][0]+fn[j][0]),axis=0),label=pretty_names[j],color=colors[j])


# ax.set_facecolor('w')
# ax.legend(prop={'size': labelsize}, loc='upper left', frameon=False,bbox_to_anchor=(1.05, 1))
ax.legend(prop={'size': labelsize}, loc='best', frameon=False)
ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.tick_params(axis='both', which='minor', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
# plt.setp(ax.xaxis.get_label(), visible=True, text='IoU')
# plt.setp(ax.get_xticklabels(), visible=True, ha='right')

# ax.grid(b=True, which='major', color='b', linestyle='-')
ax.set_ylabel('Average Precision', fontsize = labelsize)
ax.set_xlabel('IoU threshold', fontsize = labelsize)
# plt.set(xlabel='IoU threshold', ylabel='Average Precision',fontsize=labelsize)
# plt.tight_layout()
# plt.yscale('log')


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.patch.set_alpha(0.0)

fig.patch.set_facecolor(background_color)
plt.show()


fig.savefig('/home/kcutler/DataDrive/skelpose_paper/AP_vs_IoU.eps',bbox_inches=tight_bbox)
fig.savefig('/home/kcutler/DataDrive/skelpose_paper/AP_vs_IoU.png',bbox_inches=tight_bbox)
# fig.savefig('/home/kcutler/DataDrive/skelpose_paper/AP_vs_IoU_all.eps',bbox_inches="tight",pad_inches = 0.05)
# fig.savefig('/home/kcutler/DataDrive/skelpose_paper/AP_vs_IoU_all.png',bbox_inches="tight",pad_inches = 0.05)

In [ ]:
# import mpld3
# mpld3.enable_notebook()

k = -1
print(names[k])
mAP = np.mean(ap[k][0],axis=1) # AP averaged over all IoU thresholds, per image (so a vector length 133)
# ious = ap[n][0][:,1]
idx = np.argsort(mAP)
mAP = mAP[np.argsort(mAP)]
vals = mAP[mAP<0.6]
ind = idx[mAP<0.6] 
K = len(vals)
# colors = sinebow(N+1)
colors = pl.cm.jet(np.linspace(0,1,K+1))
sz = 5
fig = plt.figure(figsize=(sz, sz/golden)) 
ax = plt.axes()
for j in range(K):
    ax.plot(ap[k][0][ind[j]].T,color=colors[j+1])

plt.show()

In [ ]:
from cellpose_omni.utils import ncolorlabel
from cellpose_omni import transforms

In [ ]:
#code developing the error ideas 
ind = range(4)

from matplotlib.colors import ListedColormap
plt.style.use('dark_background')
# now find and crop specific cells for comparison 
# oversegmneted cells will have low IoUs spread out over several hits; let's take a specific image from the rpevious list as an example. We can see that there are three true labels, but 
# several predicted labels. The IoU can be at most 1, but small precited labels will always be quite small, up to 1/(area of the true label). But we don't really want to care about the 
cm2 = ListedColormap([color for color in sinebow(5).values()])

k = ind[3] #specific cmall example as a test
# k = ind[1]
k = 3
j = 2 # look at original cellpose

# r = OvR[j][k]
# mx = np.max(r,axis=0)
# hits = np.sum(r==mx,axis=1)
# # true,pred = np.nonzero(r>mx/2)
# (unique,counts) = np.unique(true,return_counts=True)
# errors = counts-1

#defer 'belonging' to maximum; only issue is if there is a spurious label with no overlap at all (e.g. dead cell)
# opposite case (e.g. edge cell): maybe should filter by area 
# mx[mx==0] = np.nan
# hits_pred = r==mx,axis=1) # sum will be zero if a cell label has zero hits, not sure which ones these are
# err = np.abs(hits-1)
# cell_errors[j][k] = err  #error if >1 or =0 
# terr = np.sum(hits[hits>1]-1)+np.sum(hits==0)
# total_errors[j] += terr



m = 0
r_pred = OvR[j][k].copy()
mx = np.max(r_pred,axis=0) 
# std = np.std(r_pred,axis=0)
mx[mx==0] = np.nan
hits_pred = r_pred==mx
err = np.abs(np.sum(hits_pred,axis=1)-1)



# err = cell_errors[j][k]# assuming proper precomputing 
areas = cell_areas[k]
error_cutoff = 1
area_cutoff = 10
err_indexes = np.nonzero(np.logical_and(err>=error_cutoff,areas>area_cutoff))[0]

# print(true)
# print(pred)
# print(counts)
# print(hits,err,terr)

# fig, axes = plt.subplots(nrows=1, ncols=5)
# fig.set_size_inches(40,40)

# axes[0].imshow(imgs[k],cmap = 'gray')
# axes[0].axis('off')
# #     plt.show()
# axes[1].imshow(ncolorlabel(masks[j][k],n=5),cmap = cm2,interpolation='nearest')
# axes[1].axis('off')

# axes[2].imshow(ncolorlabel(masks_gt[k],n=5),cmap = cm2,interpolation='nearest')
# axes[2].axis('off')

# axes[3].imshow(r,cmap = 'viridis',interpolation='nearest')
# axes[3].axis('off')

# axes[4].imshow(r==mx,cmap = 'viridis',interpolation='nearest')
# axes[4].axis('off')
# plt.show()


# err_indexes = np.nonzero(err)[0]

pad = 5

for l in err_indexes[0:10]:
    print(err[l])
    mgt = masks_gt_clean[k]
    y,x = np.nonzero(mgt==l+1)
    max_y,max_x = np.array(mgt.shape)-1
    
    y0 = max(0,min(y)-pad)
    y1 = min(max_y,max(y)+pad)
    x0 = max(0,min(x)-pad)
    x1 = min(max_x,max(x)+pad)
    
    p = transforms.normalize99(imgs[k][y0:y1,x0:x1])

    mask_gt = mgt[y0:y1,x0:x1]
    bini = mask_gt==l+1
    img0 = np.stack((p,p,p),axis=2)
    outli = transforms.normalize99(utils.outline_view(img0*255,bini))
    bin0 = np.stack((bini,bini,bini),axis=2)
    gt_pic = np.hstack((img0,outli,bin0))
    mask_pred = masks[j][k][y0:y1,x0:x1]
    flow_pred = transforms.normalize99(flows[j][k][y0:y1,x0:x1])
#                     print(bini.shape,flow_OG.shape)
#                     flow_OG = np.hstack([flow_OG[:,:,i]*bini for i in range(flow_OG.shape[-1])])
# #                     flow_OG = np.multiply(flow_OG,bini)
#                     print(flow_OG.shape)

    
    
    inds = np.where(hits_pred[l,:])
#                     print('OG inds',inds,np.unique(mask_OG))
    rmask = np.zeros_like(mask_gt)
    for i,label in enumerate(inds[0]):
        rmask[mask_pred==label+1] = i+1

    outl_pred = transforms.normalize99(utils.outline_view(img0*255,rmask))
    res_pred = np.hstack((img0,bin0,cm2(ncolorlabel(mask_pred))[:,:,:3],outl_pred,flow_pred))
    plt.figure(figsize=[2,2])
    plt.imshow(res_pred)
    plt.axis('off')
    plt.show()

# plt.imshow(r,cmap='viridis')
# print(np.sum(r,axis=1))
# plt.hist(r)
# p = plt.plot(threshold,IoU_hits[0][1],'w',threshold,IoU_hits[1][1],'g')

For this plot, I should put both the training and test errors. The training dataset has a lot more long cells in it. In fact, I should do the analysis with the train, test, and combined to show correlation and lack of overfitting. 

In [ ]:

labelsize = 7
matplotlib.rc('xtick', labelsize=labelsize) 
matplotlib.rc('ytick', labelsize=labelsize) 
# matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['figure.dpi'] = 300 # resolution for pngs 
linewidth = 0.75



fig = plt.figure(figsize=(sz, sz/golden)) 
ax = plt.axes()


# plt.minorticks_on()
# plt.xticks(np.arange(min(x), max(x)+1, .25))
# plt.xlim([0.4,1.1])
# plt.ylim([0,1])
# plt.yticks(np.arange(0, 1.1, .25))
# colors = ['r','g','b','k']
for j in [0]:
# for j in range(n):
# for j in [0,3,-1]:
    ax.scatter(np.concatenate(cell_areas),np.concatenate(cell_errors[j]),s=3,color=colors[j],label=names[j],alpha=1,edgecolors='none')
    
# ax.scatter(np.concatenate(cell_areas),np.concatenate(cell_errors),s=1)
# plt.xlim([0.05,1e5])
plt.xscale('log')
plt.xticks()
# plt.yscale('log')
plt.ylim([0,80])
# labelsize = 11
# ax.set_facecolor('w')

# ax.legend(prop={'size': labelsize}, loc='upper left',frameon=False,markerscale=3)
ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
# ax.tick_params(axis='both', which='minor', labelsize=labelsize,length=3, direction="in",colors=axcol,bottom=True,left=True)
# plt.setp(ax.xaxis.get_label(), visible=True, text='IoU')
# plt.setp(ax.get_xticklabels(), visible=True, ha='right')

# ax.grid(b=True, which='major', color='b', linestyle='-')
ax.set_ylabel('Segmentation errors', fontsize = labelsize)
ax.set_xlabel('Cell area (px$^2$)', fontsize = labelsize)
# plt.set(xlabel='IoU threshold', ylabel='Average Precision',fontsize=labelsize)
# plt.gcf().subplots_adjust(bottom=0.15)
# plt.tight_layout()

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.patch.set_alpha(0.0)

fig.patch.set_facecolor(background_color)
plt.show()

tight_bbox_raw = ax.get_tightbbox(fig.canvas.get_renderer())
a = 50
tight_bbox_raw._points+=[[-a,-a],[a,a]]
tight_bbox = mpl.transforms.TransformedBbox(tight_bbox_raw, mpl.transforms.Affine2D().scale(1./fig.dpi))

fig.savefig('/home/kcutler/DataDrive/skelpose_paper/errors_vs_area.eps',bbox_inches=tight_bbox)
fig.savefig('/home/kcutler/DataDrive/skelpose_paper/errors_vs_area.png',bbox_inches=tight_bbox)
# fig.savefig('/home/kcutler/DataDrive/skelpose_paper/errors_vs_area_loglog.eps',bbox_inches="tight")
# fig.savefig('/home/kcutler/DataDrive/skelpose_paper/errors_vs_area_linear.eps',bbox_inches="tight")

# fig.savefig('/home/kcutler/DataDrive/skelpose_paper/errors_vs_area.svg')

In [ ]:
# something pretty surprising: there aren't all that many cells that get oversegmented, and skelpose and distpose are super close - edit: no longer!
# this metric doesn't tell the whole story, but the differnet thrsholds do show there is more *bad* oversegmentation in the original cellpose
# as for the rest, it might be the case that a lot of these are edge cells, especially the longer ones

darkmode = False
if darkmode:
    plt.style.use('dark_background')
    axcol = 'w'
    colors = sinebow(n+1)
    colors = [colors[j+1] for j in range(len(names))]
else:
    mpl.rcParams.update(mpl.rcParamsDefault)
    axcol = 'k'
#     cmap = mpl.cm.get_cmap('viridis')
#     colors = cmap(np.linspace(0,.9,len(names)))
    
fig = plt.figure(figsize=(sz, sz/golden)) 
ax = plt.axes()
for j in range(n):
    x = np.concatenate(cell_errors[j],axis=0)
    ax.hist(x,bins=np.arange(0,80, 1),color = colors[j],log=True,range=(0,80),label=names[j],align='left')
    print(names[j] +' error count:',np.count_nonzero(x))
    
print('Number of cells:',x.shape[-0])
# plt.xscale('log')
ax.legend(prop={'size': labelsize},  loc='upper left', frameon=False,bbox_to_anchor=(1.05, 1),markerscale=3)

ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
plt.xticks(np.arange(0,80, 5))
ax.set_ylabel('Number of cells', fontsize = labelsize)
ax.set_xlabel('Segmentation errors', fontsize = labelsize)
plt.show()

From cell errors we can look at tons of examples. My current code just gives every single instance of an error. We see from the above histogram that my version barely gives any errors above 1 (these are issues with cell division, I think). 

- Enforce a minimum crop size 
- Sort by number of errors 
- Dump images into a PDF with info 

In [ ]:
import os
from scipy.ndimage.morphology import binary_erosion 
from cellpose_omni import utils
import cv2
from cellpose_omni.io import imsave
from PIL import Image, ImageFont, ImageDraw, ImageOps
from matplotlib.colors import ListedColormap
import os, datetime, gc, warnings, glob


plt.style.use('dark_background')
cmap = matplotlib.cm.get_cmap('viridis')
cm2 = ListedColormap([color for color in sinebow(5).values()])

basedir = '/home/kcutler/DataDrive/skelpose_paper/Comparison Examples/Per Cell GT Comparison/'
if not os.path.isdir(basedir):
    os.mkdir(basedir)
    
cutoff=1
pad = 10
N = len(masks)
nimg = len(masks_gt)
bkct = 500
bg = 0.5


txtpad = 10
yoffset = [0,0]
buffer = 15
# for k in range(nimg):
# for k in [0]:
for j in [-1]:
    pdf = FPDF(unit='pt')
    pdf.add_page()
    pdf.set_font('Arial', 'B', 8)
    pdf.set_text_color(r=255, g=255, b=255)
    pdf.set_fill_color(0,0,0)
    ph = pdf.h
    pw = pdf.w
    pdf.page_break_trigger = ph
    pdf.rect(0,0,pw,ph,'F')
    pdf.set_xy(0,0)
    ind = 0
    cnt = 0
    for k in range(nimg):
#     for k in range(50):
        file = img_names[k]
        basename = os.path.splitext(os.path.split(file)[-1])[0]

        label_list = []
        err = cell_errors[j][k]      
        err_indexes = np.nonzero(err>=cutoff)[0]
        num_errors = err_indexes.size
        if num_errors > 0:
            
            for l in err_indexes:
                name = basename + '_' + names[j] + '_cell_number_'+ str(l) + '.png'
                savepath = os.path.join(basedir, name)
#                 print(j,k,l)
                
                if not os.path.isfile(savepath):
                    mgt = masks_gt_clean[k]
                    p = transforms.normalize99(imgs[k])
                    img0 = p**(np.log(bg)/np.log(np.mean(p[binary_erosion(mgt==0)])))

                    y,x = np.nonzero(mgt==l+1)
                    max_y,max_x = np.array(mgt.shape)-1

                    y0 = max(0,min(y)-pad)
                    y1 = min(max_y,max(y)+pad)
                    x0 = max(0,min(x)-pad)
                    x1 = min(max_x,max(x)+pad)


                    p = img0[y0:y1,x0:x1]

                    pic = [[]]*2
                    img0 = np.stack((p,p,p),axis=2)
                    maski = masks[j][k][y0:y1,x0:x1]
                    flowi = transforms.normalize99(flows[j][k][y0:y1,x0:x1])
                    outli = transforms.normalize99(utils.outline_view(img0*255,maski))
                    pic[0] = np.hstack((img0,outli,flowi))

                    maski = mgt[y0:y1,x0:x1]
    #                 print(np.unique(maski),l)
                    bini = maski==l+1
                    bini = np.stack((bini,bini,bini),axis=2)
                    outli = transforms.normalize99(utils.outline_view(img0*255,maski))
                    pic[1] = np.hstack((cm2(ncolorlabel(maski))[:,:,:3],outli,bini))

                    img = np.vstack((pic[0],pic[-1]))
                
                    io.imsave(savepath,np.uint8(img*255))
                im = Image.open(savepath)
                y = pdf.get_y()
                
                h = ((pw/2)/im.size[0])*im.size[1]
                if yoffset[ind]+h+txtpad+buffer>=ph:
                    pdf.add_page()
                    pdf.rect(0,0,pw,ph,'F')
                    pdf.set_xy(0,0)
                    yoffset = [0,0]
                    ind = 0
                    pdf.set_xy(0,0)
                    cnt = 0
                elif cnt==0 or cnt==1:
                    pdf.set_xy(ind*pw/2,yoffset[ind])
                else:
                    pdf.set_xy(ind*pw/2,yoffset[ind]+txtpad)
                
                if h<ph:
                    pdf.image(savepath,w=pw/2)
                else:
                    pdf.image(savepath,h=ph-2*txtpad)
                pdf.cell(30, 10, txt=name)
#                 pdf.cell(30, 10, txt=str(j)+','+str(k)+','+str(l))
#                 pdf.cell(30, txtpad, str(ind))
#                 print(ind,yoffset)
                yoffset[ind] = pdf.get_y()
                ind = (ind+1)%2
                cnt += 1
                
    pdf.output('/home/kcutler/DataDrive/'+names[j]+'_errors.pdf','F')
                


In [ ]:
import os
from scipy.ndimage.morphology import binary_erosion 
from cellpose_omni import utils, transforms
from cellpose_omni.utils import ncolorlabel
import cv2
from cellpose_omni.io import imsave
from PIL import Image, ImageFont, ImageDraw, ImageOps
from matplotlib.colors import ListedColormap
import os, datetime, gc, warnings, glob


plt.style.use('dark_background')
cmap = matplotlib.cm.get_cmap('viridis')
cm2 = ListedColormap([color for color in sinebow(5).values()])

basedir = '/home/kcutler/DataDrive/skelpose_paper/Comparison Examples/newcompare2/'
if not os.path.isdir(basedir):
    os.mkdir(basedir)

area_cutoff = 20
error_cutoff=2
pad = 10
N = len(masks)
nimg = len(masks_gt)
bkct = 500
bg = 0.5
ext = '.png'

cnt = 0

txtpad = 10
yoffset = [0,0]
buffer = 15
# for k in range(nimg):
# for k in [0]:
for j in [0]:
    ind = 0
    for k in range(nimg):
#     for k in range(50):
        file = img_names[k]
        basename = os.path.splitext(os.path.split(file)[-1])[0]

        label_list = []
        err = cell_errors[j][k]
        areas = cell_areas[k]
        err_indexes = np.nonzero(np.logical_and(err>=error_cutoff,areas>area_cutoff))[0]
        num_errors = err_indexes.size
        
        m = 2
        r_OG = OvR[j][k] 
        mx = np.max(r_OG,axis=0) 
        std = np.std(r_OG,axis=0)
        mx[mx==0] = np.nan
        hits_OG = r_OG>=(mx-m*std)
#         hits_OG = r_OG>=0.2
        
        r_new = OvR[-1][k] 
        mx = np.max(r_new,axis=0) 
        std = np.std(r_new,axis=0)
        mx[mx==0] = np.nan
        hits_new = r_new>=(mx-m*std)
#         hits_new = r_new>= 0.2
        
        if num_errors > 0:
            
            
            for l in err_indexes:
                name = basename + '_' + names[j] + '_cell_number_'+ str(l)
                savepath = os.path.join(basedir, name + ext)
                
                overwrite = True
                if not os.path.isfile(savepath) or overwrite:
                    mgt = masks_gt_clean[k]
                    p = transforms.normalize99(imgs[k])
                    img0 = p**(np.log(bg)/np.log(np.mean(p[binary_erosion(mgt==0)])))

                    y,x = np.nonzero(mgt==l+1)
                    max_y,max_x = np.array(mgt.shape)-1

                    y0 = max(0,min(y)-pad)
                    y1 = min(max_y,max(y)+pad)
                    x0 = max(0,min(x)-pad)
                    x1 = min(max_x,max(x)+pad)

                    p = img0[y0:y1,x0:x1]

                    mask_gt = mgt[y0:y1,x0:x1]
    #                 print(np.unique(maski),l)
                    bini = mask_gt==l+1
                    img0 = np.stack((p,p,p),axis=2)
                    outli = transforms.normalize99(utils.outline_view(img0*255,bini))
                    bin0 = np.stack((bini,bini,bini),axis=2)
#                     bin0 = np.zeros_like(img0)
                    gt_pic = np.hstack((img0,outli,bin0))
                    mask_OG = masks[j][k][y0:y1,x0:x1]
                    flow_OG = transforms.normalize99(flows[j][k][y0:y1,x0:x1])
#                     print(bini.shape,flow_OG.shape)
#                     flow_OG = np.hstack([flow_OG[:,:,i]*bini for i in range(flow_OG.shape[-1])])
# #                     flow_OG = np.multiply(flow_OG,bini)
#                     print(flow_OG.shape)


                    inds = np.where(hits_OG[l,:])
#                     print('OG inds',inds,np.unique(mask_OG))
                    rmask = np.zeros_like(mask_gt)
                    for i,label in enumerate(inds[0]):
                        rmask[mask_OG==label+1] = i+1
                
                    outl_OG = transforms.normalize99(utils.outline_view(img0*255,rmask))
#                     OG_pic = np.hstack((cm2(ncolorlabel(mask_OG))[:,:,:3],outl_OG,flow_OG))
                    
                    mask_new =  masks[-1][k][y0:y1,x0:x1]
                    flow_new = transforms.normalize99(flows[-1][k][y0:y1,x0:x1])
                    inds = np.where(hits_new[l,:])
#                     print('new inds',inds,np.unique(mask_new))
                    rmask = np.zeros_like(mask_gt)
                    for i,label in enumerate(inds[0]):
                        rmask[mask_new==label+1] = i+1
                    
                    bin_new = np.stack((rmask>0,rmask>0,rmask>0),axis=2)
                    outl_new = transforms.normalize99(utils.outline_view(img0*255,rmask))
#                     new_pic = np.hstack((cm2(ncolorlabel(mask_new))[:,:,:3],outl_new,flow_new))
                    

#                     img = np.vstack((gt_pic,OG_pic,new_pic))
#                     plt.imshow(img)
#                     plt.axis('off')
#                     plt.show()
                    
#                     x = range(len(r_OG[l,:]))
#                     x2 = range(len(r_new[l,:]))
#                     plt.plot(x,r_OG[l,:],'r',x2,r_new[l,:],'w')
#                     plt.show()
                    
                
#                     io.imsave(savepath,np.uint8(img*255))
                    savedir = os.path.join(basedir, name)
                    if not os.path.isdir(savedir):
                        os.mkdir(savedir)
                    io.imsave(os.path.join(savedir,'phase'+ext),np.uint8(img0*255))
                    io.imsave(os.path.join(savedir,'outlines_GT'+ext),np.uint8(outli*255))
                    io.imsave(os.path.join(savedir,'outlines_OG'+ext),np.uint8(outl_OG*255))
                    io.imsave(os.path.join(savedir,'flow_OG'+ext),np.uint8(flow_OG*255))
                    io.imsave(os.path.join(savedir,'flow_OG_isolated'+ext),np.uint8(bin0*flow_OG*255))
                    io.imsave(os.path.join(savedir,'binary_mask'+ext),np.uint8(bin0*255))
                    io.imsave(os.path.join(savedir,'outlines_new'+ext),np.uint8(outl_new*255))
                    io.imsave(os.path.join(savedir,'flow_new'+ext),np.uint8(flow_new*255))
                    io.imsave(os.path.join(savedir,'flow_new_isolated'+ext),np.uint8(bin_new*flow_new*255))
                
                    if p.shape[0]<p.shape[1]:
                            stack = np.vstack
                    else:
                        stack  = np.hstack
                    
                    stack_OG = stack((img0,outl_OG,flow_OG))
                    stack_new = stack((img0,outl_new,flow_new))
                    stack_OG_isolated = stack((img0,outl_OG,bin0*flow_OG))
                    stack_new_isolated = stack((img0,outl_new,bin_new*flow_new))
#                     plt.imshow(new_stack)
#                     plt.show()
                    io.imsave(os.path.join(savedir,'stack_OG'+ext),np.uint8(stack_OG*255))
                    io.imsave(os.path.join(savedir,'stack_new'+ext),np.uint8(stack_new*255))
        
                    io.imsave(os.path.join(savedir,'stack_OG_isolated'+ext),np.uint8(stack_OG_isolated*255))
                    io.imsave(os.path.join(savedir,'stack_new_isolated'+ext),np.uint8(stack_new_isolated*255))
                

#                 im = Image.open(savepath)


                


In [ ]:
masks_cmp = io.imread('/home/kcutler/DataDrive/merge_ftsN/xy1/edited_labels.tif')

In [ ]:
from cellpose_omni import utils
n = len(masks_cmp)
print(n)
diam_old = []
diam_new = []

x = range(n)
for k in x:#random.sample(range(0, n), 2):

    m = masks_cmp[k]
    fastremap.renumber(m,in_place=True)
    diam_old.append(utils.diameters(m,skel=False)[0])
    diam_new.append(utils.diameters(m,skel=True)[0])

In [ ]:
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

from cellpose_omni.utils import sinebow
master_color_scheme = [[i,0,0] for i in np.linspace(1,.5,3)]+[[i,i,i] for i in np.linspace(.75,0,n-3)]
golden = (1 + 5 ** 0.5) / 2
sz = 2.5
labelsize = 7

%matplotlib inline
darkmode = False
if darkmode:
    plt.style.use('dark_background')
    axcol = 'w'
    colors = sinebow(n+1)
    colors = [colors[j+1] for j in range(n)]
    background_color = 'k'
else:
    mpl.rcParams.update(mpl.rcParamsDefault)
    axcol = 'k'
    cmap = mpl.cm.get_cmap('viridis')
#     colors = cmap(np.linspace(0,.9,len(names)))
    colors = master_color_scheme
    background_color = np.array([242,242,242])/255
    
mpl.rcParams['figure.dpi'] = 300

fig = plt.figure(figsize=(sz, sz/golden)) 
ax = plt.axes()

plt.plot(range(n),diam_old,'r',range(n),diam_new,'k')

ax.legend(prop={'size': labelsize}, loc='best', frameon=False)
ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.tick_params(axis='both', which='minor', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.set_ylabel('Diameter Metric', fontsize = labelsize)
ax.set_xlabel('Frame number', fontsize = labelsize)
# plt.ylim([0,100])
fig.patch.set_facecolor(background_color)
plt.show()

name = '/home/kcutler/DataDrive/skelpose_paper/diameter metric_ftsN_comparison'
fig.savefig(name+'.eps',bbox_inches='tight')
fig.savefig(name+'.png',bbox_inches='tight')